In [1]:
import numpy as np

In [2]:
# Set the seed for the numpy random number generator.
# This ensures that we get reproducible results
np.random.seed(67129)

## Q1 Generating a sequence of states from a 2nd order Markov process

We're going to generate a sequence from a 2nd order discrete Markov process. First of all we'll need to define our transition probability matrix. We'll use the one given in the text. It is non-square. We'll call this the longform of the transition probability matrix, because it is longer than it is wide.

In [3]:
### Define our rate matrix
rate_matrix_longform = np.array([[0.27, 0.33, 0.4],
                                 [0.12, 0.67, 0.21],
                                 [0.3, 0.3, 0.4],
                                 [0.08, 0.56, 0.36],
                                 [0.72, 0.19, 0.09],
                                 [0.43, 0.27, 0.3],
                                 [0.16, 0.16, 0.68],
                                 [0.45, 0.45, 0.1],
                                 [0.25, 0.38, 0.37]])

Since we're going to want to generate lots of sequences in Q2 we'll define a function to generate a sequence which we can call multiple times. We'll pass the transition rate matrix, and the starting states of the sequence into this function. The function will loop through the positions in the state sequence, each time picking the row in the transition probability matrix corresponding to the preceding two states in the sequence and then using the probability distribution in that row to sample the next state in the sequence.

In [4]:
# Define function to sample a state sequence
def generate_state_sequence(rate_matrix, seq_len, state0=0, state1=0):
    '''
    Function to generate a state sequence of a specified length
    given the rate matrix and the first two states of the sequence
    
    :param rate_matrix: The transition probability matrix for 
                        our 2nd order Markov process.
    :type rate_matrix: A numpy array of shape (n_state**2, n_state)
                       where n_state is the number of possible state values.
                       
    :param seq_len: The length of the state sequence to be generated.
    :type seq_len: int
    
    :param state0: The state value of the first state in the sequence.
    :type state0: int
    
    :param state1: The state value of the second state in the sequence.
    :type state1: int
    
    :return: The sampled state sequence.
    :rtype: A numpy array of integers.
    '''

    # Extract the number of state values
    n_state = rate_matrix.shape[1]
    
    # Initialize the state sequence and populate the first 
    # two state values
    state_sequence = np.full(seq_len, 0, dtype=np.int16)
    state_sequence[0] = state0
    state_sequence[1] = state1

    # Loop over the remaining positions (timepoints) along the sequence
    for idx in range(2, seq_len):
        # Calculate which row of the transition rate matrix we need
        l = n_state*state_sequence[idx-2] + state_sequence[idx-1]
        
        # Sample from the probability distribution in the selected row 
        # of the transition probability matrix
        state_sequence[idx] = np.random.choice(a=np.arange(start=0,stop=n_state,step=1), p=rate_matrix_longform[l,:])

    return state_sequence

Now we'll call the function to generate a 200 length sequence

In [5]:
# Call our state sequence generating function
state_sequence = generate_state_sequence(rate_matrix_longform, seq_len = 200)

# Print the first 20 state values
print(state_sequence[0:20])

[0 0 0 2 1 2 1 2 1 1 0 1 1 0 1 1 2 0 2 0]


## Q2 Estimate the limiting distribution

We'll generate 10000 sequences of length 200 using our function defined in Q1. We'll sample the first two states of each sequence independently from the uniform distribution. From each generated sequence we'll record the final state of the sequence and we'll consider this as a sample from the limiting distribution. Once we have our 10000 quasi-samples of state from the limiting distribution estimating the limiting distribution is just a case of calculating the proportions the different state values in our 10000 samples.

In [6]:
# Set the number of sequences we want to generate
n_sequence = 10000

# Set the sequence length required
seq_len = 200

# Extract the number of possible state values
n_state = rate_matrix_longform.shape[1]

# Initialize an array to count the occurrences of 
# each end state value
end_state_counts = np.zeros(n_state)

# Loop over the generated sequences
for i in range(n_sequence):
    
    # Print out progress, as this could take a while
    if i % 500 == 0:
        print("Generating sequence " , i)
        
    # Generate the starting states of the sequence
    state0 = np.random.randint(low=0, high=n_state)
    state1 = np.random.randint(low=0, high=n_state)
    sequence = generate_state_sequence(rate_matrix=rate_matrix_longform, seq_len=seq_len, state0=state0, state1=state1)
    
    # Extract the end state value of the generated sequence and update 
    # the corresponding count
    end_state_counts[sequence[seq_len-1]] += 1.0

# Now convert the end-state counts to proportions and 
# print out the result
print(" ")
print("The estimate of the limiting distribution:")
print(end_state_counts/n_sequence)

Generating sequence  0
Generating sequence  500
Generating sequence  1000
Generating sequence  1500
Generating sequence  2000
Generating sequence  2500
Generating sequence  3000
Generating sequence  3500
Generating sequence  4000
Generating sequence  4500
Generating sequence  5000
Generating sequence  5500
Generating sequence  6000
Generating sequence  6500
Generating sequence  7000
Generating sequence  7500
Generating sequence  8000
Generating sequence  8500
Generating sequence  9000
Generating sequence  9500
 
The estimate of the limiting distribution:
[0.3027 0.3826 0.3147]


## Q3 Finding the stationary distribution

In [7]:
## First we'll convert our transition probability matrix 
## into square form

# Extract the number of possible state values
n_state = rate_matrix_longform.shape[1]

rate_matrix_square = np.zeros((n_state**2,n_state**2))

for i in range(n_state):
    for j in range(n_state):
        for k in range(n_state):
            l = i*n_state + j
            l_prime = j*n_state + k
            rate_matrix_square[l,l_prime] = rate_matrix_longform[l,k]

Let's check what the rate matrix looks like to check that it is as we expect.

In [8]:
rate_matrix_square

array([[0.27, 0.33, 0.4 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.12, 0.67, 0.21, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.3 , 0.3 , 0.4 ],
       [0.08, 0.56, 0.36, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.72, 0.19, 0.09, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.43, 0.27, 0.3 ],
       [0.16, 0.16, 0.68, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.45, 0.45, 0.1 , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.38, 0.37]])

Now we'll calculate the left eigenvectors and eigenvalues of the square rate matrix

In [9]:
# Calculate the left eigenvectors and eigenvalues of the square rate matrix
# We'll do this by calculating the right eigenvectors (and eigenvalues) of the transpose 
# of our square rate matrix
rate_eigen_2nd_order = np.linalg.eig(np.transpose(rate_matrix_square))

Now look at the eigenvalues we've just computed

In [10]:
rate_eigen_2nd_order[0]

array([ 1.        +0.j        , -0.25149675+0.52394605j,
       -0.25149675-0.52394605j, -0.28060319+0.j        ,
        0.3240332 +0.j        ,  0.23321847+0.j        ,
        0.18002103+0.j        , -0.061838  +0.08168315j,
       -0.061838  -0.08168315j])

We can see the first eigenvalue is 1, indicating a stationary distribution, so we'll extract the corresponding eigenvector and normalize it so that its elements sum to 1. This will be our estimate of the stationary distribution. 

In [11]:
# Extract the first eigenvector (we can drop the zero imaginary part)
stationary_distribution_composite = np.real(rate_eigen_2nd_order[1][:,0])

# Rescale the vector so its elements sum to 1
stationary_distribution_composite /=np.sum(stationary_distribution_composite)

In [12]:
stationary_distribution_composite

array([0.03956221, 0.12589135, 0.14206479, 0.1749071 , 0.15957343,
       0.05077819, 0.09304905, 0.09979393, 0.11437996])

This looks a bit strange. It doesn't look like the limiting distribution we estimated in Q2. This is because our estimate of the stationary distribution is for our composite states $ij$. We need to sum up over all the values of $i$. We'll do that summation by creating an 9x3 matrix and using matrix multiplication 

In [13]:
# Add up over all values $i$ from the composite stationary distribution elements indexed by $ij$.  
s_matrix = np.vstack((np.identity(n_state), np.identity(n_state), np.identity(n_state)))
stationary_distribution = np.matmul(stationary_distribution_composite, s_matrix)

In [14]:
stationary_distribution

array([0.30751836, 0.38525871, 0.30722293])

To double check, it shouldn't matter whether we add up over all the values of $i$ in the composite state probabilities $\pi_{ij}$ or whether we add up over all the values of $j$. We should get the same estimate for our stationary distribution, so let's calculate it this second way

In [15]:
np.matmul(np.transpose(s_matrix), stationary_distribution_composite)

array([0.30751836, 0.38525871, 0.30722293])

Yes, we get the same estimate for the stationary distribution. We can also see that the estimate of the stationary distribution is the same as our estimate of the limiting distribution (to within the numerical precision with which we have estimated the limiting distribution). 